In [2]:
from tensorflow import keras
import pandas as pd
import numpy as np
import scipy
import os
import pickle
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import optimizers

In [3]:
df = pd.read_csv("./data/features_30_sec.csv")
df.head()
df.columns


Index(['filename', 'length', 'chroma_stft_mean', 'chroma_stft_var', 'rms_mean',
       'rms_var', 'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'harmony_mean', 'harmony_var', 'perceptr_mean', 'perceptr_var', 'tempo',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var',
  

In [4]:
df.corr()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
length,1.000000,0.017426,0.073979,0.101801,0.116661,0.052175,0.121994,0.062503,0.118993,0.040363,...,-0.019677,0.025483,0.039179,0.003457,-0.025331,0.014822,0.010576,0.045884,0.014803,0.050338
chroma_stft_mean,0.017426,1.000000,-0.461729,0.499926,0.261544,0.615728,0.285926,0.517710,0.041088,0.598122,...,0.332359,-0.452955,-0.163474,-0.456298,0.269935,-0.478452,-0.089509,-0.501768,0.281965,-0.548577
chroma_stft_var,0.073979,-0.461729,1.000000,-0.078777,0.290138,-0.246441,0.305819,-0.036896,0.346907,-0.168721,...,-0.311178,0.361752,0.205117,0.389468,-0.239597,0.378200,0.184323,0.365291,-0.231715,0.333581
rms_mean,0.101801,0.499926,-0.078777,1.000000,0.607399,0.510154,0.324077,0.530017,0.099830,0.537902,...,0.083936,-0.160347,-0.029562,-0.163490,0.118239,-0.164834,0.031297,-0.207680,0.124413,-0.252434
rms_var,0.116661,0.261544,0.290138,0.607399,1.000000,0.347831,0.583353,0.402333,0.320793,0.368570,...,-0.093893,0.091863,0.168169,0.100234,-0.002585,0.058050,0.186953,0.049879,0.029364,0.012087
spectral_centroid_mean,0.052175,0.615728,-0.246441,0.510154,0.347831,1.000000,0.520548,0.904435,0.011256,0.979637,...,0.153741,-0.180200,0.027473,-0.158666,0.228275,-0.136272,0.027918,-0.146316,0.269601,-0.205531
spectral_centroid_var,0.121994,0.285926,0.305819,0.324077,0.583353,0.520548,1.000000,0.622834,0.636494,0.545128,...,-0.186864,0.072319,0.175573,0.094155,-0.063059,0.059431,0.188945,0.046847,-0.021837,0.008115
spectral_bandwidth_mean,0.062503,0.517710,-0.036896,0.530017,0.402333,0.904435,0.622834,1.000000,0.220159,0.956189,...,0.027206,-0.146152,0.105010,-0.122739,0.114873,-0.102737,0.098217,-0.121584,0.187786,-0.193574
spectral_bandwidth_var,0.118993,0.041088,0.346907,0.099830,0.320793,0.011256,0.636494,0.220159,1.000000,0.053632,...,-0.267098,0.104727,0.128638,0.108537,-0.178183,0.074145,0.124554,0.028696,-0.157001,0.026121
rolloff_mean,0.040363,0.598122,-0.168721,0.537902,0.368570,0.979637,0.545128,0.956189,0.053632,1.000000,...,0.132443,-0.180777,0.039693,-0.157343,0.205222,-0.131517,0.048248,-0.144641,0.256570,-0.215384


In [5]:
df = df.drop(labels=["filename","length","harmony_mean","harmony_var","perceptr_mean","perceptr_var","tempo"],axis=1)

In [9]:
from sklearn.preprocessing import StandardScaler
fit = StandardScaler()
X = fit.fit_transform(np.array(df.iloc[:,:-1],dtype="float"))


import pickle
pickle_open = open("fit.pkl","wb")
pickle.dump(fit, pickle_open)
pickle_open.close()

In [7]:
df.iloc[:,:-1].columns

Index(['chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var',
       'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var'],
      dtype='object')

In [28]:
class_list = df.iloc[:,-1]
converter = LabelEncoder()
y= converter.fit_transform(class_list)
integer_mapping = {l: i for i, l in enumerate(converter.classes_)}
integer_mapping

import pickle
pickle_open = open("converter.pkl","wb")
pickle.dump(converter, pickle_open)
pickle_open.close()

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
def init_model(model , optimizer, epochs):
    batch_size = 128
    model.compile(optimizer=optimizers.Adam(lr=1e-04),loss="sparse_categorical_crossentropy" , metrics="accuracy")
    return model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=epochs)

In [10]:
model = keras.Sequential([
    keras.layers.Dense(512,activation="relu",input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(256,activation="relu"),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(128,activation="relu"),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(64,activation="relu"),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(10, activation="softmax")
])

init_model(model, "adam", 600)

c:\users\naman mistry\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/600
7/7 [==============================] - 2s 32ms/step - loss: 2.6930 - accuracy: 0.1037 - val_loss: 2.3176 - val_accuracy: 0.1200
Epoch 2/600
7/7 [==============================] - 0s 9ms/step - loss: 2.6209 - accuracy: 0.0938 - val_loss: 2.2981 - val_accuracy: 0.1100
Epoch 3/600
7/7 [==============================] - 0s 9ms/step - loss: 2.4999 - accuracy: 0.1125 - val_loss: 2.2821 - val_accuracy: 0.1600
Epoch 4/600
7/7 [==============================] - 0s 8ms/step - loss: 2.4791 - accuracy: 0.1025 - val_loss: 2.2688 - val_accuracy: 0.2000
Epoch 5/600
7/7 [==============================] - 0s 8ms/step - loss: 2.4906 - accuracy: 0.1187 - val_loss: 2.2571 - val_accuracy: 0.2400
Epoch 6/600
7/7 [==============================] - 0s 9ms/step - loss: 2.4502 - accuracy: 0.0850 - val_loss: 2.2465 - val_accuracy: 0.2850
Epoch 7/600
7/7 [==============================] - 0s 9ms/step - loss: 2.4345 - accuracy: 0.0988 - val_loss: 2.2372 - val_accuracy: 0.3150
Epoch 8/600
7/7 [=========

In [12]:
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 5ms/step - loss: 0.7477 - accuracy: 0.7600


[0.7477031946182251, 0.7599999904632568]

In [13]:
songname = "./data/genres_original/rock/rock.00001.wav"
y, sr = librosa.load(songname, mono=True, duration=30)

In [14]:
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rms = librosa.feature.rms(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)

In [15]:
chroma_stft_mean = np.mean(chroma_stft)
chroma_stft_var = np.var(chroma_stft)
rms_mean = np.mean(rms)
rms_var = np.var(rms)
spec_cent_mean = np.mean(spec_cent)
spec_cent_var = np.var(spec_cent)
spec_bw_mean = np.mean(spec_bw)
spec_bw_var = np.var(spec_bw)
rolloff_mean = np.mean(rolloff)
rolloff_var = np.var(rolloff)
zcr_mean = np.mean(zcr)
zcr_var= np.var(zcr)
mfcc_list = []
for i in range(0,20):
    mfcc_list.append(np.mean(mfcc[i]))
    mfcc_list.append(np.var(mfcc[i]))

In [16]:
df.iloc[:,:-1].columns

Index(['chroma_stft_mean', 'chroma_stft_var', 'rms_mean', 'rms_var',
       'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_bandwidth_mean', 'spectral_bandwidth_var', 'rolloff_mean',
       'rolloff_var', 'zero_crossing_rate_mean', 'zero_crossing_rate_var',
       'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean', 'mfcc2_var', 'mfcc3_mean',
       'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_mean', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc7_var', 'mfcc8_mean',
       'mfcc8_var', 'mfcc9_mean', 'mfcc9_var', 'mfcc10_mean', 'mfcc10_var',
       'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean', 'mfcc12_var', 'mfcc13_mean',
       'mfcc13_var', 'mfcc14_mean', 'mfcc14_var', 'mfcc15_mean', 'mfcc15_var',
       'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean', 'mfcc17_var', 'mfcc18_mean',
       'mfcc18_var', 'mfcc19_mean', 'mfcc19_var', 'mfcc20_mean', 'mfcc20_var'],
      dtype='object')

In [17]:
final_features = [chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spec_cent_mean,spec_cent_var,spec_bw_mean,spec_bw_var,rolloff_mean,rolloff_var,zcr_mean,zcr_var]+mfcc_list

In [18]:
len(final_features)

final_features

[0.36552483,
 0.092101134,
 0.18054315,
 0.0049171033,
 1529.3454638808878,
 227181.30332113683,
 1845.4233876403491,
 118365.91916115051,
 3271.6635500314437,
 1363362.4448733404,
 0.054981526944659444,
 0.0008506006341843945,
 -111.1363,
 3155.0442,
 134.15948,
 746.50037,
 -24.647953,
 318.57535,
 41.54462,
 203.63092,
 -3.057123,
 131.22209,
 28.483068,
 73.23623,
 -10.571674,
 146.63039,
 14.921676,
 53.475594,
 -14.384826,
 119.06498,
 8.840185,
 49.770504,
 -4.5424523,
 71.27923,
 9.939368,
 62.046318,
 -4.6650004,
 54.043236,
 3.6263325,
 34.682495,
 -5.541119,
 49.429386,
 3.3648012,
 28.2074,
 -4.94252,
 38.26377,
 -1.4728665,
 38.031345,
 0.10088544,
 31.619509,
 -5.3325586,
 37.253326]

In [19]:
final_features = np.array(final_features)
final_features = fit.transform([final_features])
final_features

array([[-0.16110974,  0.7451806 ,  0.75572356,  0.51358859, -0.93967727,
        -0.60521803, -0.75490016, -0.19410575, -0.82584653, -0.3376805 ,
        -1.16341625, -0.70666739,  0.33277029, -0.21408243,  1.10506089,
         0.09022732, -0.72545141, -0.52164169,  0.31532597, -0.14474498,
        -0.15654908, -0.42874775,  1.17035355, -0.78894151, -0.54779753,
         0.51846722,  0.45921654, -0.84634059, -0.89262827,  0.77260472,
         0.13996635, -0.87056364,  0.21700942, -0.11207859,  0.81455994,
        -0.20314806,  0.02122289, -0.41730351,  0.36845319, -0.87362554,
        -0.34320593, -0.39673966,  0.48433976, -0.9632313 , -0.21473528,
        -0.7282751 , -0.51214784, -0.74687832,  0.64720186, -0.93154227,
        -1.10485361, -0.72717888]])

In [20]:
print(model.predict(final_features.reshape(1,52)))

[[0.1304506  0.0047557  0.16067743 0.08375132 0.01013119 0.03129735
  0.06061276 0.00402648 0.01701214 0.49728492]]


In [21]:
(X_test[0])

array([ 0.45415235, -0.96333601,  0.65481871, -0.21442785,  0.19656154,
       -0.53141412, -0.24617751, -0.67735263,  0.01515206, -0.57699313,
        0.83327186, -0.25386623,  1.09671953, -0.92857794,  0.22162514,
       -0.49969816, -1.48256695, -0.27885264, -0.15771986, -0.8324527 ,
       -2.13549948, -0.76598242,  0.57224818, -0.54446945, -1.62873761,
       -0.12035316,  0.37244698, -1.07117442, -2.50973435,  0.54443445,
        1.0492596 , -0.08466683, -1.87797681, -0.79603894,  0.17462611,
       -0.70303736, -1.54041795, -0.67077808, -0.19485562, -0.41278848,
       -1.3685084 , -0.62180584,  0.71941436, -0.21872174, -0.7692418 ,
       -0.67695005,  0.4041564 , -0.17381702, -0.1515229 , -0.68432441,
        0.19088737, -0.70679426])

In [22]:
converter.inverse_transform([np.argmax(model.predict(final_features.reshape(1,52)))])

array(['rock'], dtype=object)

In [23]:
converter.inverse_transform([y_test[1]])

array(['disco'], dtype=object)

In [23]:
X_test[1]

array([-0.06515107, -0.22168739,  0.04045049, -0.53034622,  0.09785694,
       -0.57065476,  0.10431699, -0.79294427,  0.24537744, -0.64895843,
        0.11234201, -0.57826281,  0.59711899, -0.8492732 ,  0.0451501 ,
       -0.89292593, -0.3273999 , -1.09861651,  0.92760639, -0.38571154,
       -1.30495609, -0.4815797 ,  0.67112007, -1.04108194, -1.52632753,
       -0.52118142,  0.84085041, -0.68719811, -1.50841743, -0.43640523,
        0.9591122 ,  0.07758962, -1.72002774, -0.45155584,  0.61248268,
       -0.71252813, -1.65527912, -0.2750914 ,  0.92954542, -0.87011718,
       -1.31609035, -0.49755353,  0.46691702, -0.61821089, -1.22815799,
       -0.44061673,  0.91937292, -0.47237905, -0.86778117, -0.42233057,
        0.10339899, -0.24341258])

In [24]:
model.save("model.h5")